# प्रॉम्प्ट इन्जिनियरिङको परिचय

प्रॉम्प्ट इन्जिनियरिङ भनेको प्राकृतिक भाषा प्रशोधन कार्यहरूको लागि प्रॉम्प्टहरू डिजाइन र अनुकूलन गर्ने प्रक्रिया हो। यसमा सही प्रॉम्प्टहरू चयन गर्ने, तिनीहरूको प्यारामिटरहरू मिलाउने, र तिनीहरूको प्रदर्शन मूल्याङ्कन गर्ने काम समावेश हुन्छ। प्रॉम्प्ट इन्जिनियरिङ उच्च शुद्धता र कार्यक्षमताका लागि अत्यन्तै महत्वपूर्ण छ, विशेष गरी NLP मोडेलहरूमा। यस खण्डमा, हामी OpenAI मोडेलहरू प्रयोग गरेर प्रॉम्प्ट इन्जिनियरिङका आधारभूत कुरा बुझ्नेछौं।


### अभ्यास १: टोकनाइजेसन
टोकनाइजेसनको अन्वेषण गर्नुहोस् tiktoken प्रयोग गरेर, जुन OpenAI बाट खुला स्रोत छिटो टोकनाइजर हो
थप उदाहरणहरूको लागि [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) हेर्नुहोस्।


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### अभ्यास २: Github Models कुञ्जी सेटअप प्रमाणित गर्नुहोस्

तपाईंको Github Models अन्तर्क्रिया सही रूपमा सेटअप भएको छ कि छैन भनेर जाँच्न तलको कोड चलाउनुहोस्। यो कोडले एउटा साधारण प्रॉम्प्ट प्रयोग गर्छ र completion सही छ कि छैन भनेर जाँच गर्छ। इनपुट `oh say can you see` को completion लगभग `by the dawn's early light..` जस्तो आउनुपर्छ।


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### अभ्यास ३: बनावटी कुरा

LLM लाई यस्तो विषयमा completion फर्काउन अनुरोध गर्दा के हुन्छ भनेर अन्वेषण गर्नुहोस्, जुन विषय सायद अस्तित्वमा छैन, वा यस्तो विषय जसको बारेमा उसलाई थाहा नहुन सक्छ किनभने त्यो उसको पूर्व-प्रशिक्षित डेटासेटभन्दा बाहिर (अधिक हालको) छ। यदि तपाईं फरक प्रॉम्प्ट वा फरक मोडेल प्रयोग गर्नुहुन्छ भने जवाफ कसरी परिवर्तन हुन्छ हेर्नुहोस्।


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### अभ्यास ४: निर्देशनमा आधारित
"text" भेरिएबल प्रयोग गरेर मुख्य सामग्री सेट गर्नुहोस्
र "prompt" भेरिएबल प्रयोग गरेर सो मुख्य सामग्रीसँग सम्बन्धित निर्देशन दिनुहोस्।

यहाँ हामीले मोडेललाई दोस्रो कक्षाका विद्यार्थीका लागि पाठलाई संक्षेपमा भन्न अनुरोध गरेका छौं।


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### अभ्यास ५: जटिल प्रॉम्प्ट
एक अनुरोध प्रयास गर्नुहोस् जसमा system, user र assistant सन्देशहरू छन्
System ले assistant को सन्दर्भ सेट गर्छ
User र Assistant सन्देशहरूले बहुपल्ट संवादको सन्दर्भ दिन्छन्

ध्यान दिनुहोस् कि system सन्दर्भमा assistant को व्यक्तित्व "व्यङ्ग्यात्मक" सेट गरिएको छ।
अर्को व्यक्तित्व सन्दर्भ प्रयोग गरेर प्रयास गर्नुहोस्। वा फरक इनपुट/आउटपुट सन्देशहरूको श्रृंखला प्रयास गर्नुहोस्


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### अभ्यास: आफ्नो अन्तर्ज्ञान अन्वेषण गर्नुहोस्
माथिका उदाहरणहरूले तपाईंलाई नयाँ प्रॉम्प्टहरू (साधारण, जटिल, निर्देशन आदि) बनाउनका लागि प्रयोग गर्न सकिने ढाँचाहरू दिन्छन् - हामीले चर्चा गरेका अन्य विचारहरू जस्तै उदाहरणहरू, संकेतहरू र थपका लागि अन्य अभ्यासहरू सिर्जना गरेर अन्वेषण गर्न प्रयास गर्नुहोस्।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। मूल भाषा मा रहेको मूल दस्तावेज़लाई नै आधिकारिक स्रोत मान्नुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
